In [14]:
!pip install Flask keras tensorflow pyngrok

In [34]:
!pip install pyngrok

In [16]:
!ls

dog_cat_classifier  sample_data


In [27]:
from google.colab import files
uploaded = files.upload()

Saving pathmodel.h5 to pathmodel (1).h5


In [57]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


In [58]:
app = Flask(__name__)

In [59]:
model = load_model('pathmodel.h5')

In [60]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def classify_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    return 'dog' if prediction[0][0] > 0.5 else 'cat'


In [63]:
from pyngrok import ngrok

In [74]:
!ngrok authtoken 2jEsPzf639Wdp2EFpeDqjRkvamr_7CW5emBByF7rE3WJ6efMN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [81]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from pyngrok import ngrok

app = Flask(__name__)
model = load_model('pathmodel.h5') # Make sure to replace 'pathmodel.h5' with the actual path to your model

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def preprocess_image(img_path, target_size=(256, 256)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def classify_image(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    return 'dog' if prediction[0][0] > 0.5 else 'cat'

@app.route('/')
def home():
    return '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Dog or Cat Classifier</title>
        <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f0f0;
            margin: 20px;
        }

        h1 {
            text-align: center;
            color: #333;
        }

        form {
            margin: 20px auto;
            width: 80%;
            max-width: 600px;
            padding: 20px;
            background-color: #fff;
            border: 1px solid #ddd;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }

        input[type="file"] {
            margin-bottom: 10px;
        }

        input[type="submit"] {
            background-color: #4CAF50;
            color: white;
            border: none;
            padding: 10px 20px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            border-radius: 5px;
            cursor: pointer;
        }

        #result {
            margin-top: 20px;
            padding: 10px;
            background-color: #f0f0f0;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
    </style>
    </head>
    <body>
      <h1>Upload an Image to Classify</h1>
      <form id="uploadForm" action="/upload" method="post" enctype="multipart/form-data">
          <input type="file" id="fileInput" name="file" accept="image/*">
          <input type="submit" value="Upload">
      </form>
      <div id="result"></div>
      <script>
          const form = document.getElementById('uploadForm');
          const fileInput = document.getElementById('fileInput');
          const resultDiv = document.getElementById('result');

          form.addEventListener('submit', async (event) => {
              event.preventDefault();
              const formData = new FormData(form);
              const response = await fetch('/upload', {
                  method: 'POST',
                  body: formData
              });
              const result = await response.json();
              resultDiv.innerText = `Prediction: ${result.result}`;
          });
      </script>
    </body>
    </html>
  '''

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return "No file part"
    file = request.files['file']
    if file.filename == '':
        return "No selected file"
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        # Make sure this path is correct for your environment
        filepath = os.path.join('uploads', filename)
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        file.save(filepath)
        result = classify_image(filepath)
        return jsonify({'result': result})

if __name__ == '__main__':
    # Start Ngrok tunnel
    public_url = ngrok.connect(5000) # Change 'port' to an integer
    print(f"Ngrok Tunnel: {public_url}")

    # Run Flask app
    app.run(host='0.0.0.0', port=5000)

Ngrok Tunnel: NgrokTunnel: "https://3a90-34-19-20-122.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Jul/2024 14:27:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jul/2024 14:27:44] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 270ms/step


INFO:werkzeug:127.0.0.1 - - [14/Jul/2024 14:27:52] "POST /upload HTTP/1.1" 200 -


1/1 [==============================] - 0s 64ms/step


INFO:werkzeug:127.0.0.1 - - [14/Jul/2024 14:28:48] "POST /upload HTTP/1.1" 200 -
